In [1]:
# Threat Zone Prediction Model Evaluation

# This notebook evaluates our trained models on test data and explores their performance in real-world scenarios.

In [2]:
pip install shapely

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pyproj

Note: you may need to restart the kernel to use updated packages.


In [4]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, roc_curve, roc_auc_score,
    precision_recall_curve, average_precision_score
)
import joblib

# Add parent directory to path for importing modules
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
print("Current working dir:", os.getcwd())
print("Project root added to sys.path:", os.path.abspath(os.path.join(os.getcwd(), '..')))


from models.threat_model import ThreatModel
from models.explosion_model import ExplosionModel
from models.dispersion_model import DispersionModel
from models.preprocessing import generate_synthetic_data
from utils.geo_utils import calculate_threat_zone

from config import Config

%matplotlib inline
plt.style.use('ggplot')
sns.set_theme(style="whitegrid")


Current working dir: c:\Users\Nidhi\model\notebooks
Project root added to sys.path: c:\Users\Nidhi\model


In [5]:
## Load Models and Test Data
from utils.visualization import _polygon_to_coordinates

In [9]:
# Load the models
threat_model = ThreatModel('../models/saved/threat_model.joblib')
explosion_model = ExplosionModel('../models/saved/explosion_model.joblib')
dispersion_model = DispersionModel('../models/saved/dispersion_model.joblib')

# Generate test data if it doesn't exist
try:
    test_data = pd.read_csv('../data/test/test.csv')
    print(f"Loaded test data with {len(test_data)} samples")
except FileNotFoundError:
    print("Test data not found, generating synthetic test data")
    X, y = generate_synthetic_data(n_samples=1000, include_anomalies=True)
    test_data = X.copy()
    test_data['threat_level'] = y

    # Create directory if it doesn't exist
    os.makedirs('../data/test', exist_ok=True)
    test_data.to_csv('../data/test/test.csv', index=False)
    print(f"Generated {len(test_data)} test data samples")


d:\condA\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator GradientBoostingRegressor from version 1.3.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
Error loading explosion model: No module named 'sklearn.ensemble._gb_losses'


ValueError: y should be a 1d array, got an array of shape (100, 5) instead.

In [ ]:
## Evaluate Threat Detection Model


In [ ]:
# Separate features and target
X_test = test_data.drop(columns=['threat_level'])
y_test = test_data['threat_level']

# Make predictions using the threat model
predictions = []
scores = []
detailed_results = []

for _, row in X_test.iterrows():
    result = threat_model.predict(
        row['mq2'], row['mq4'], row['mq6'], row['mq8'],
        row['temperature'], row['humidity']
    )
    # Store prediction (1 for high or medium risk, 0 for low risk or safe)
    prediction = 1 if result['risk_score'] >= 0.5 else 0
    predictions.append(prediction)
    scores.append(result['risk_score'])
    detailed_results.append(result)

# Calculate metrics
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions, zero_division=0)
recall = recall_score(y_test, predictions, zero_division=0)
f1 = f1_score(y_test, predictions, zero_division=0)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Display classification report
print("\nClassification Report:")
print(classification_report(y_test, predictions))


In [ ]:
# Plot confusion matrix
cm = confusion_matrix(y_test, predictions)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Safe', 'Threat'],
            yticklabels=['Safe', 'Threat'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Plot ROC curve
fpr, tpr, _ = roc_curve(y_test, scores)
roc_auc = roc_auc_score(y_test, scores)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()


In [ ]:
# Plot Precision-Recall curve
precision_curve, recall_curve, _ = precision_recall_curve(y_test, scores)
ap = average_precision_score(y_test, scores)

plt.figure(figsize=(8, 6))
plt.plot(recall_curve, precision_curve, color='blue', lw=2,
         label=f'Precision-Recall curve (AP = {ap:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="upper right")
plt.grid(True)
plt.show()


In [ ]:
## Risk Score Distribution


In [ ]:
# Add scores to test data
test_data_with_scores = test_data.copy()
test_data_with_scores['risk_score'] = scores

# Plot risk score distribution by actual threat level
plt.figure(figsize=(10, 6))
sns.histplot(data=test_data_with_scores, x='risk_score', hue='threat_level',
             bins=50, kde=True, element="step")
plt.axvline(x=0.5, color='red', linestyle='--', label='Decision Threshold (0.5)')
plt.title('Risk Score Distribution by Actual Threat Level')
plt.xlabel('Risk Score')
plt.ylabel('Count')
plt.legend(['Decision Threshold (0.5)', 'Safe', 'Threat'])
plt.grid(True)
plt.show()


In [ ]:
## Test Explosion Model


In [ ]:
# Test explosion model with different gas concentrations
gas_concentrations = [500, 1000, 2000, 5000]
temperatures = [20, 30, 40, 50]

results = []

for gas in gas_concentrations:
    for temp in temperatures:
        explosion_result = explosion_model.predict(gas, temp)

        # Store key parameters
        results.append({
            'gas_concentration': gas,
            'temperature': temp,
            'energy_release': explosion_result['energy_release'],
            'fireball_radius': explosion_result['fireball_radius'],
            'distance_to_overpressure_15kPa': explosion_result['distance_to_overpressure']['15kPa'],
            'distance_to_radiation_10kW': explosion_result['distance_to_radiation']['10kW/m²']
        })

# Convert to DataFrame for easier analysis
explosion_results_df = pd.DataFrame(results)
explosion_results_df.head()


In [ ]:
# Visualize relationship between gas concentration, temperature, and explosion effects
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot energy release
for temp in temperatures:
    temp_data = explosion_results_df[explosion_results_df['temperature'] == temp]
    axes[0, 0].plot(temp_data['gas_concentration'], temp_data['energy_release'],
                   marker='o', label=f'Temperature: {temp}°C')
axes[0, 0].set_title('Energy Release vs Gas Concentration')
axes[0, 0].set_xlabel('Gas Concentration (ppm)')
axes[0, 0].set_ylabel('Energy Release (MJ)')
axes[0, 0].legend()
axes[0, 0].grid(True)

# Plot fireball radius
for temp in temperatures:
    temp_data = explosion_results_df[explosion_results_df['temperature'] == temp]
    axes[0, 1].plot(temp_data['gas_concentration'], temp_data['fireball_radius'],
                   marker='o', label=f'Temperature: {temp}°C')
axes[0, 1].set_title('Fireball Radius vs Gas Concentration')
axes[0, 1].set_xlabel('Gas Concentration (ppm)')
axes[0, 1].set_ylabel('Fireball Radius (m)')
axes[0, 1].legend()
axes[0, 1].grid(True)

# Plot distance to overpressure
for temp in temperatures:
    temp_data = explosion_results_df[explosion_results_df['temperature'] == temp]
    axes[1, 0].plot(temp_data['gas_concentration'], temp_data['distance_to_overpressure_15kPa'],
                   marker='o', label=f'Temperature: {temp}°C')
axes[1, 0].set_title('Distance to 15kPa Overpressure vs Gas Concentration')
axes[1, 0].set_xlabel('Gas Concentration (ppm)')
axes[1, 0].set_ylabel('Distance (m)')
axes[1, 0].legend()
axes[1, 0].grid(True)

# Plot distance to radiation
for temp in temperatures:
    temp_data = explosion_results_df[explosion_results_df['temperature'] == temp]
    axes[1, 1].plot(temp_data['gas_concentration'], temp_data['distance_to_radiation_10kW'],
                   marker='o', label=f'Temperature: {temp}°C')
axes[1, 1].set_title('Distance to 10kW/m² Radiation vs Gas Concentration')
axes[1, 1].set_xlabel('Gas Concentration (ppm)')
axes[1, 1].set_ylabel('Distance (m)')
axes[1, 1].legend()
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()
